In [2]:
import pandas as pd
import json
import requests
import math

In [230]:
#Global paramenters
#Name of drug changes
base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
base_url_2 = "https://rxnav.nlm.nih.gov/REST/"
getAllRelatedInfo = "/allrelated.json"
getRelatedByType = "/related.json?tty=IN"
df = pd.DataFrame()


In [239]:
df = pd.read_json("https://rxnav.nlm.nih.gov/REST/rxcui/6754/related.json?tty=IN")

In [240]:
df.relatedGroup.conceptGroup

[{'tty': 'IN',
  'conceptProperties': [{'rxcui': '6754',
    'name': 'meperidine',
    'synonym': '',
    'tty': 'IN',
    'language': 'ENG',
    'suppress': 'N',
    'umlscui': ''}]}]

In [221]:
df = pd.DataFrame(df.relatedGroup.conceptGroup)

In [223]:
df = pd.DataFrame(df.conceptProperties[0])

In [224]:
df

,rxcui,name,synonym,tty,language,suppress,umlscui
0,7052,morphine,,IN,ENG,N,


In [236]:
def query_self(name_of_drug):
    rd_df = pd.read_json(base_url+name_of_drug+getRelatedByType)
    rd_df = pd.DataFrame(rd_df.relatedGroup.conceptGroup)
    rd_df = pd.DataFrame(rd_df.conceptProperties[0])
    return rd_df

In [237]:
query_self("7052")

,rxcui,name,synonym,tty,language,suppress,umlscui
0,7052,morphine,,IN,ENG,N,


In [186]:
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+name_of_drug+getAllRelatedInfo)
    rd_df = pd.DataFrame(rd_df.allRelatedGroup.conceptGroup)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
        row += 1
    return df

In [215]:
query_drug('4337').to_csv("test.csv")

In [188]:
fentanyl = "4337"
morphine = "7052"
buprenorphine = "1819"
pethidine = "6754"
isonipecaine = "6754"
meperidine = "6754"
dihydromorphinone = "3423"
hydromorphone = "3423"
methadone = "6813"
hydrocodone = "5489"
tapentadol = "787390"
oxycodone = "7804"
oxymorphone = "7814"
codeine = "2670"
dihydrocodeine = "23088"

In [238]:
#loop array to get complete list
df_opioid = pd.DataFrame()
drugs = [fentanyl, morphine, buprenorphine, pethidine, isonipecaine, meperidine, dihydromorphinone, hydromorphone, methadone, hydrocodone, tapentadol, oxycodone, oxymorphone, codeine, dihydrocodeine]
for drug in drugs:
    self_df = query_self(drug)
    merg_df = query_drug(drug)
    df_opioid = pd.concat([df_opioid, self_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
    df_opioid = pd.concat([df_opioid, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_opioid

,rxcui,name,synonym,tty,language,suppress,umlscui
0,4337,fentanyl,,IN,ENG,N,
1,1053648,Abstral,,BN,ENG,N,
2,1115547,Lazanda,,BN,ENG,N,
3,1237051,Subsys,,BN,ENG,N,
4,151678,Duragesic,,BN,ENG,N,
...,...,...,...,...,...,...,...
2262,1155445,chlorpheniramine / dihydrocodeine / phenylephr...,,SCDG,ENG,N,
2263,1158923,dihydrocodeine Injectable Product,,SCDG,ENG,N,
2264,1158924,dihydrocodeine Oral Liquid Product,,SCDG,ENG,N,
2265,1158925,dihydrocodeine Oral Product,,SCDG,ENG,N,


In [241]:
#Export
df_opioid.to_excel(r"C:/Users/Daniel/Documents/opiod_list/opioid_list/New_Complete_Opioids_List.xlsx", index=False)

In [242]:
df_opioid

,rxcui,name,synonym,tty,language,suppress,umlscui
0,4337,fentanyl,,IN,ENG,N,
1,1053648,Abstral,,BN,ENG,N,
2,1115547,Lazanda,,BN,ENG,N,
3,1237051,Subsys,,BN,ENG,N,
4,151678,Duragesic,,BN,ENG,N,
...,...,...,...,...,...,...,...
2262,1155445,chlorpheniramine / dihydrocodeine / phenylephr...,,SCDG,ENG,N,
2263,1158923,dihydrocodeine Injectable Product,,SCDG,ENG,N,
2264,1158924,dihydrocodeine Oral Liquid Product,,SCDG,ENG,N,
2265,1158925,dihydrocodeine Oral Product,,SCDG,ENG,N,


In [245]:
#take list and format it to the RXNorm api format
#USING TTY
#This list was made by cobing the excel files
df_list = pd.read_excel("C:/Users/Daniel/Documents/opiod_list/opioid_list/Complete_Opioids_List.xlsx")
df_list = df_list[['Code', 'Description']]
df_list.rename(columns= {'Code' : 'rxcui', 'Description': 'name'}, inplace = True)
df_list

,rxcui,name
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...
...,...,...
5152,996712,codeine phosphate 20 MG / guaifenesin 400 MG /...
5153,996716,codeine phosphate 20 MG / guaifenesin 400 MG /...
5154,996727,codeine phosphate 20 MG / guaifenesin 400 MG O...
5155,996730,codeine phosphate 20 MG / pseudoephedrine hydr...


In [246]:
#This list was created by pulling from the RxNorm API
df_opioid = df_opioid[['rxcui', 'name']]

In [247]:
df_opioid

,rxcui,name
0,4337,fentanyl
1,1053648,Abstral
2,1115547,Lazanda
3,1237051,Subsys
4,151678,Duragesic
...,...,...
2262,1155445,chlorpheniramine / dihydrocodeine / phenylephr...
2263,1158923,dihydrocodeine Injectable Product
2264,1158924,dihydrocodeine Oral Liquid Product
2265,1158925,dihydrocodeine Oral Product


In [248]:
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame({"drugs_in_list_rxcui": [], "drugs_in_list_name": []})
new_drug = pd.DataFrame({"new_drugs_rxcui": [], "new_drugs_name": []})
for code in df_opioid['rxcui']:
    drug_name = df_opioid['name'].where(df_opioid['rxcui'] == code).dropna().values[0]
    if df_list.isin([int(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = same_drug.append({'drugs_in_list_rxcui': code, 'drugs_in_list_name' : drug_name}, ignore_index = True)
    else:
        new_drug_count += 1
        new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  1815
New drug count:  452


In [249]:
df_final = pd.DataFrame()

In [250]:
#Displays same and new drugs
df_final = same_drug.join(new_drug)
df_final.head()

,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,4337,fentanyl,1649572,Cartridge
1,1053648,Abstral,1649574,Injection
2,1115547,Lazanda,1797831,Metered Dose Nasal Spray
3,1237051,Subsys,316949,Injectable Solution
4,151678,Duragesic,316987,Transdermal System


In [251]:
absent_drugs = pd.DataFrame({"drugs_not_found_rxcui": [], "drugs_not_found_name": []})
count = 0
for code in df_list['rxcui']:
    drug_name = df_list['name'].where(df_list['rxcui'] == code).dropna().values[0]
    if df_final.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = absent_drugs.append({'drugs_not_found_rxcui': str(code), 'drugs_not_found_name': drug_name}, ignore_index = True)
print('Count: ', count)
absent_drugs.head()
        
        

Count:  1815


,drugs_not_found_rxcui,drugs_not_found_name
0,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...
1,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...
2,1114110,codeine phosphate 1.6 MG/ML / guaifenesin 40 M...
3,1114878,codeine phosphate 1.6 MG/ML / pseudoephedrine ...
4,1655058,meperidine hydrochloride 150 MG Oral Tablet


In [252]:
df_final = pd.DataFrame()
df_final = absent_drugs.join(same_drug)
df_final = df_final.join(new_drug)
print(df_final.shape) #check if correct size
df_final.head()

(3342, 6)


,drugs_not_found_rxcui,drugs_not_found_name,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,4337,fentanyl,1649572,Cartridge
1,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,1053648,Abstral,1649574,Injection
2,1114110,codeine phosphate 1.6 MG/ML / guaifenesin 40 M...,1115547,Lazanda,1797831,Metered Dose Nasal Spray
3,1114878,codeine phosphate 1.6 MG/ML / pseudoephedrine ...,1237051,Subsys,316949,Injectable Solution
4,1655058,meperidine hydrochloride 150 MG Oral Tablet,151678,Duragesic,316987,Transdermal System


In [253]:
#export
df_final.to_excel("C:/Users/Daniel/Documents/opiod_list/opioid_list/Opioids_Comparison.xlsx", index=False)

In [254]:
df_final

,drugs_not_found_rxcui,drugs_not_found_name,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,4337,fentanyl,1649572,Cartridge
1,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,1053648,Abstral,1649574,Injection
2,1114110,codeine phosphate 1.6 MG/ML / guaifenesin 40 M...,1115547,Lazanda,1797831,Metered Dose Nasal Spray
3,1114878,codeine phosphate 1.6 MG/ML / pseudoephedrine ...,1237051,Subsys,316949,Injectable Solution
4,1655058,meperidine hydrochloride 150 MG Oral Tablet,151678,Duragesic,316987,Transdermal System
...,...,...,...,...,...,...
3337,860239,guaifenesin 10 MG/ML / hydrocodone bitartrate ...,NaN,NaN,NaN,NaN
3338,860530,guaifenesin 20 MG/ML / hydrocodone bitartrate ...,NaN,NaN,NaN,NaN
3339,891172,guaifenesin 20 MG/ML / hydrocodone bitartrate ...,NaN,NaN,NaN,NaN
3340,996462,codeine phosphate 2 MG/ML / guaifenesin 20 MG/...,NaN,NaN,NaN,NaN


In [255]:
#start here for absolete
#reduce data to be used to find obsolete
not_found = df_final[['drugs_not_found_rxcui', 'drugs_not_found_name']]
not_found.rename(columns= {'drugs_not_found_rxcui' : 'rxcui', 'drugs_not_found_name': 'name'}, inplace = True)
not_found

C:\Users\Daniel\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,rxcui,name
0,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...
1,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...
2,1114110,codeine phosphate 1.6 MG/ML / guaifenesin 40 M...
3,1114878,codeine phosphate 1.6 MG/ML / pseudoephedrine ...
4,1655058,meperidine hydrochloride 150 MG Oral Tablet
...,...,...
3337,860239,guaifenesin 10 MG/ML / hydrocodone bitartrate ...
3338,860530,guaifenesin 20 MG/ML / hydrocodone bitartrate ...
3339,891172,guaifenesin 20 MG/ML / hydrocodone bitartrate ...
3340,996462,codeine phosphate 2 MG/ML / guaifenesin 20 MG/...


In [258]:
df_status = pd.DataFrame()
index = 0
for code in not_found['rxcui']:
    getRxcuiHistoryStatus = 'rxcui/{}/historystatus.json'.format(code)
    df_type = pd.read_json(base_url_2+getRxcuiHistoryStatus)
    df_type = pd.DataFrame(df_type.rxcuiStatusHistory.metaData, index=[0])
    df_type['rxcui'] = code
    df_status = pd.concat([df_status, df_type]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)

In [259]:
df_status.tail(30)

,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate,rxcui
3312,Active,RXNORM,102011,,YES,102011,,,1187691
3313,Active,RXNORM,112011,,YES,112011,,,1190641
3314,Active,RXNORM,112011,,YES,112011,,,1190646
3315,Active,RXNORM,052012,,YES,052012,,,1250693
3316,Active,RXNORM,082012,,YES,082012,,,1302741
3317,Obsolete,RXNORM,102012,,NO,102012,122021,,1308440
3318,Obsolete,RXNORM,072015,,NO,072015,042021,,1650980
3319,Obsolete,RXNORM,072015,,NO,072015,042021,,1650981
3320,Active,RXNORM,022016,,YES,022016,,,1729581
3321,Active,RXNORM,022016,,YES,022016,,,1729586


In [260]:
df_not_found = not_found.merge(df_status, on='rxcui')

In [267]:
df_not_found

,rxcui,name,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate
0,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,Obsolete,RXNORM,082011,,NO,082011,122021,
1,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,Obsolete,RXNORM,082011,,NO,082011,032022,
2,1114110,codeine phosphate 1.6 MG/ML / guaifenesin 40 M...,Obsolete,RXNORM,082011,,NO,082011,122021,
3,1114878,codeine phosphate 1.6 MG/ML / pseudoephedrine ...,Obsolete,RXNORM,082011,,NO,082011,122021,
4,1655058,meperidine hydrochloride 150 MG Oral Tablet,Obsolete,RXNORM,072015,,NO,072015,082021,
...,...,...,...,...,...,...,...,...,...,...
3337,860239,guaifenesin 10 MG/ML / hydrocodone bitartrate ...,Obsolete,RXNORM,092009,,NO,092009,122021,
3338,860530,guaifenesin 20 MG/ML / hydrocodone bitartrate ...,Obsolete,RXNORM,092009,,NO,092009,122021,
3339,891172,guaifenesin 20 MG/ML / hydrocodone bitartrate ...,Obsolete,RXNORM,022010,,NO,022010,032022,
3340,996462,codeine phosphate 2 MG/ML / guaifenesin 20 MG/...,Obsolete,RXNORM,072010,,NO,072010,032022,


In [265]:
df_active=df_not_found.loc[df_not_found['status']== 'Active'].reset_index(drop=True)

In [266]:
df_active

,rxcui,name,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate
0,197873,levorphanol tartrate 2 MG Oral Tablet,Active,RXNORM,042005,,YES,052006,,
1,200240,paregoric 0.4 MG/ML Oral Solution,Active,RXNORM,042005,,YES,052006,,
2,312104,belladonna alkaloids 16.2 MG / opium 30 MG Rec...,Active,RXNORM,042005,,YES,052006,,
3,312107,belladonna alkaloids 16.2 MG / opium 60 MG Rec...,Active,RXNORM,042005,,YES,052006,,
4,830196,opium tincture 100 MG/ML Oral Solution,Active,RXNORM,022009,,YES,022009,,
...,...,...,...,...,...,...,...,...,...,...
3023,1302741,butorphanol 10 MG/ML Injectable Solution [Dolo...,Active,RXNORM,082012,,YES,082012,,
3024,1729581,remifentanil 1 MG Injection [Ultiva],Active,RXNORM,022016,,YES,022016,,
3025,1729586,remifentanil 2 MG Injection [Ultiva],Active,RXNORM,022016,,YES,022016,,
3026,1729712,remifentanil 5 MG Injection [Ultiva],Active,RXNORM,022016,,YES,022016,,


In [268]:
#export
df_not_found.to_excel("C:/Users/Daniel/Documents/opiod_list/opioid_list/Status_of_Not_found.xlsx", index=False)
df_active.to_excel("C:/Users/Daniel/Documents/opiod_list/opioid_list/Active_but_Not_Found.xlsx", index=False)